Initial script used to onboard rollcall data. Handles the following:
- Query voteview directly for underlying json data
- Convert to pandas
- Drop metadata columns (urls, clerk rollnumber, etc.)
- Drop observations that are nan
    - We require a vote type, vote result, crs_policy_area, rollnumber, vote question, and which congress voted
    - Essentially this is stating that we need to know what was voted on, by whom, and what the result was
- We then filter to observations we care about
    - not interested in who voted to stand in the 12th congress of 1824
    - We care about votes that directly correlate to laws
        - Votes need to either pass, amend, table, or override a veto
# TODO: dnelson2, finish cleaning this up

In [88]:
import pandas as pd
import requests
import numpy as np
from collections import defaultdict

In [89]:
import requests

congress = 118
chamber = "house"
rollnumber = 45

# query the raw data source
url = f"https://voteview.com/static/data/out/rollcalls/HSall_rollcalls.json"
response = requests.get(url)

if response.status_code == 200:
    data = response.json()
else:
    print("Error:", response.status_code)

In [107]:
# create initial dataframe
df = pd.DataFrame.from_records(data).drop(columns=["clerk_rollnumber", "source_documents"]).drop(columns=["congress_url"])
print(df.columns)
print(df.shape)

Index(['nominate_log_likelihood', 'vote_desc', 'clausen_codes', 'yea_count',
       'session', 'bill_number', 'dtl_desc', 'nominate_mid_1', 'issue_codes',
       'nay_count', 'congress', 'nominate_mid_2', 'rollnumber', 'date',
       'crs_subjects', 'chamber', 'peltzman_codes', 'nominate_spread_1',
       'crs_policy_area', 'nominate_spread_2', 'vote_result', 'vote_question'],
      dtype='object')
(112461, 22)


In [108]:
df["crs_policy_area"].unique().size # , df["crs_subjects"].unique().size

33

In [109]:
df["crs_policy_area"].fillna("Missing").value_counts()

crs_policy_area
Missing                                        80123
Economics and Public Finance                    7612
Congress                                        3969
Armed Forces and National Security              2863
Government Operations and Politics              2045
International Affairs                           1917
Taxation                                        1158
Energy                                          1118
Health                                          1041
Public Lands and Natural Resources               965
Transportation and Public Works                  961
Crime and Law Enforcement                        943
Finance and Financial Sector                     845
Education                                        797
Agriculture and Food                             679
Labor and Employment                             649
Environmental Protection                         598
Foreign Trade and International Finance          548
Science, Technology, Communica

In [110]:
# baseline cleaning -- require an indentifier to work with
df = df.dropna(subset=["congress", "bill_number", "rollnumber", "vote_question", "vote_desc", "crs_policy_area"])
df["congress"] = df["congress"].astype(int)
df["bill_number"] = df["bill_number"].astype(str).str.lower().str.strip()
df["vote_question"] = df["vote_question"].astype(str).str.lower().str.strip()
df["rollnumber"] = df["rollnumber"].astype(int)
print(df.shape)

(17968, 22)


In [111]:
# find the different types of bills
df["bill_type"] = df["bill_number"].str.replace(r'\d+', '', regex=True)
df["bill_type"].value_counts().head()

bill_type
hr         8618
s          4068
hres       3334
sconres     773
hconres     479
Name: count, dtype: int64

In [112]:
# these are canonical bills, the other bill types are resolutions, treaties, procedural and nomations
bill_types = ["hr", "s", "hj", "hjr", "hjres", "hjre", "sj", "sjr", "sjres", "sjre"]
df = df.loc[df["bill_type"].isin(bill_types)]
df.shape

(13256, 23)

In [113]:
# Remove any rows that are unusable -- you need to have a bill, a vote, and a location of the vote
df = df.dropna(subset=["yea_count", "nay_count", "chamber"], how="any")
print(df.shape)

(13256, 23)


In [114]:
# we now need to determine which type of votes to include
# clean vote questions
vote_questions = df["vote_question"].dropna().unique()
print(f"{len(vote_questions)} number of different vote questions")
top_100 = df["vote_question"].value_counts().head(100)
print(f"Filtering to the top 100 vote types keeps {top_100.sum() / df.shape[0]:.2%} of data")
top_100.head(20)

345 number of different vote questions
Filtering to the top 100 vote types keeps 97.97% of data


vote_question
on the amendment                                       2612
on passage                                             1830
on the motion to table                                 1505
on motion to suspend the rules and pass, as amended     940
on motion to suspend the rules and pass                 918
on motion to recommit with instructions                 838
on the motion                                           820
on passage of the bill                                  692
on the cloture motion                                   681
on agreeing to the conference report                    353
on the conference report                                258
on motion to instruct conferees                         233
suspend the rules and pass, as amended                  152
on cloture on the motion to proceed                     120
on the joint resolution                                 100
on the motion to proceed                                 86
on motion that the committ

In [115]:
vote_type_groups = {
  # this is a vote in favor of the amendment
  "amend": [
    "on agreeing to the amendment",
    "on the amendment",
    "on agreeing to the amendments en bloc",
    "on agreeing to the substitute amendment",
    "on agreeing to the amendments",
    "on agreeing to the amendment, as amended",
    "on agreeing to the senate amendment",
    "on agreeing to the amendment, as modified",
    "whether the amendment is germane"
  ],
  # this is a passage of the bill
  "pass": [
    "on passage",
    "on passage of the bill",
    "passage, objections of the president notwithstanding",
    "passage, objections of the president not withstanding",
    "passage, objections of the president to the contrary notwithstanding",
    "passage, objections ofthe president notwithstanding",
    "passage, objection of the president notwithstanding"
  ],
  # tabling is way to kill a bill -- a vote yes here is equivalent to voting no on the bill
  "table": [
    "on the motion to table",
    "table motion to reconsider",
  ],
  # suspending rules means that the bill is fast tracked to a vote -- it is essentially a vote in favor of the bill
  "suspend": [
    "on motion to suspend the rules and pass",
    "on motion to suspend the rules and pass, as amended",
    "suspend the rules and pass, as amended",
    "suspend the rules and pass",
    "suspend the rules and pass as amended",
    "suspend the rules and agree to senate amendment",
    "motion to suspend the rules and pass, as amended",
    "suspend the rules and agree to senate amendments",
    "suspend rules and pass, as amended",
    "suspend rules and passas amended",
    "motion to suspend the rules and pass",
    "suspend the rules and concur in the senate amendment",
    "suspend the rules and agree to the senate amendment",
    "suspend the rules and agree to conference report",
    "on motion to suspend rules and pass",
    "on motion to suspend rules and pass, as amended"
  ],
  # requesting changes to the bill -- tatic used to stall or kill the bill
  "recommit": [
    "on motion to recommit with instructions",
    "on motion to recommit",
    "on the motion to recommit",
    "on motion to commit with instructions",
    "on motion to recommit the conference report",
    "recommit conference report with instructions",
    "recommit the conference report with instructions",
    "on motion to recommit conference report with instructions",
    "motion to recommit conference report with instructions",
    "on motion to commit"
  ],
  # end debate a proceed to the vote -- often indicates a willingness to vote yes
  "cloture": [
    "on the cloture motion",
    "on cloture on the motion to proceed",
  ],
  # agreeing to the bill as passed by the other house -- voting yes
  "conference": [
    "on agreeing to the conference report",
    "on the conference report",
    "on motion to suspend the rules and agree to the conference report",
  ],
  # enact the bill regardless of the presidential veto -- strongly voting yes
  "veto": [
    "on overriding the veto",
    "on presidential veto"
  ],
  # accept changes made by the senate -- voting yes
  "concur": [
    "on motion to suspend the rules and concur in the senate amendment",
    "on motion to concur in the senate amendment",
    "on motion to concur in the senate amendment with an amendment",
    "on motion to suspend the rules and concur in the senate amendments",
    "on motion to concur in the senate amendments",
    "on motion to concur in senate amendments",
    "on motion to agree to the senate amendment",
    "on motion to concur in the senate adt to the house adt to the senate adt",
    "on motion to concur in the senate amdt to the house amdt to the senate amdt",
    "agree to senate amendments"
  ]
}

In [116]:
valid_vote_questions = []
for vote_type, vote_questions in vote_type_groups.items():
    valid_vote_questions.extend(vote_questions)

df = df.loc[df["vote_question"].isin(valid_vote_questions)]
df.shape

(11324, 23)

In [117]:
vote_question_to_vote_type = defaultdict(None)
for vote_type, vote_questions in vote_type_groups.items():
    vote_question_to_vote_type.update({vote_question: vote_type for vote_question in vote_questions})

In [118]:
df["vote_type"] = df["vote_question"].replace(vote_question_to_vote_type)
df = df.drop(columns="vote_question")
df["vote_type"].value_counts()

vote_type
amend         2645
pass          2551
suspend       2153
table         1520
recommit       926
cloture        801
conference     615
concur          82
veto            31
Name: count, dtype: int64

In [119]:
df["vote_result"].value_counts()

vote_result
Passed                                        4258
Amendment Rejected                            1310
Amendment Agreed to                           1302
Motion to Table Agreed to                     1241
Failed                                        1146
Bill Passed                                    684
Cloture Motion Agreed to                       342
Cloture Motion Rejected                        339
Motion to Table Failed                         264
Conference Report Agreed to                    257
Cloture on the Motion to Proceed Rejected       62
Cloture on the Motion to Proceed Agreed to      58
Veto Sustained                                  14
Amendment Not Germane                           10
Veto Overridden                                 10
Motion to Recommit Rejected                     10
Bill Defeated                                    8
Amendment Germane                                6
Agreed to                                        2
Conference Report R

In [120]:
vote_results = df["vote_result"].unique()
passed_results = [res for res in vote_results if any(res_type in res.lower() for res_type in ["passed", "agreed", "overridden", "germane"])]
failed_results = [res for res in vote_results if res not in passed_results]
passed_results, failed_results

(['Passed',
  'Amendment Agreed to',
  'Bill Passed',
  'Motion to Table Agreed to',
  'Conference Report Agreed to',
  'Cloture Motion Agreed to',
  'Amendment Germane',
  'Amendment Not Germane',
  'Veto Overridden',
  'Cloture on the Motion to Proceed Agreed to',
  'Agreed to'],
 ['Failed',
  'Amendment Rejected',
  'Motion to Recommit Rejected',
  'Motion to Table Failed',
  'Veto Sustained',
  'Cloture Motion Rejected',
  'Cloture on the Motion to Proceed Rejected',
  'Bill Defeated',
  'Conference Report Rejected'])

In [121]:
df["vote_passed"] = df["vote_result"].isin(passed_results).fillna(False)
df["vote_passed"].value_counts()

vote_passed
True     8170
False    3154
Name: count, dtype: int64

In [122]:
df.shape

(11324, 24)

In [123]:
df["date"] = pd.to_datetime(df["date"])
df = df.set_index(["date", "congress", "session", "bill_number"])

In [124]:
# look for duplicates -- are they valid
idx = df.reset_index().groupby(["date", "congress", "session", "bill_number", "vote_type"])["vote_passed"].count().idxmax()
idx

(Timestamp('2010-03-24 00:00:00'),
 np.int64(111),
 np.float64(2.0),
 'hr4872',
 'table')

In [125]:
df.loc[idx[:-1]]

/var/folders/y7/rdkd1_7d3b9fjykc4g2vvjl00000gn/T/ipykernel_89924/2540153771.py:1: PerformanceWarning: indexing past lexsort depth may impact performance.
  df.loc[idx[:-1]]


nominate_log_likelihood  \
date       congress session bill_number                            
2010-03-24 111      2.0     hr4872                        -3.663   
                            hr4872                        -5.067   
                            hr4872                        -8.462   
                            hr4872                        -4.106   
                            hr4872                        -3.043   
                            hr4872                        -1.088   
                            hr4872                        -3.663   
                            hr4872                        -3.393   
                            hr4872                        -1.088   
                            hr4872                        -1.088   
                            hr4872                        -5.599   
                            hr4872                       -12.491   
                            hr4872                        -3.385   
                            hr4872                        -1.099   
                            hr4872                        -8.059   
                            hr4872                        -6.316   
                            hr4872                        -0.280   

                                                                                 vote_desc  \
date       congress session bill_number                                                      
2010-03-24 111      2.0     hr4872       To prevent Medicare from being used for new en...   
                            hr4872       To eliminate the sweetheart deals for Tennesse...   
                            hr4872       An Act to provide for reconciliation pursuant ...   
                            hr4872       An Act to provide for reconciliation pursuant ...   
                            hr4872       To ensure that Americans can keep the coverage...   
                            hr4872       An Act to provide for reconciliation pursuant ...   
                            hr4872       An Act to provide for reconciliation pursuant ...   
                            hr4872       To reduce the cost of providing federally fund...   
                            hr4872       To protect the right of States to opt out of a...   
                            hr4872       To improve the bill by waiving the $40,000 pen...   
                            hr4872       To ensure that no State experiences a net job ...   
                            hr4872       An Act to provide for reconciliation pursuant ...   
                            hr4872                       To strike the medical device tax.   
                            hr4872       To exclude pediatric devices for persons with ...   
                            hr4872       To protect access for America's wounded warriors.   
                            hr4872       To protect the integrity of Department of Vete...   
                            hr4872       To repeal the government takeover of health care.   

                                            clausen_codes  yea_count dtl_desc  \
date       congress session bill_number                                         
2010-03-24 111      2.0     hr4872       [Social Welfare]         56     None   
                            hr4872       [Social Welfare]         54     None   
                            hr4872       [Social Welfare]         56     None   
                            hr4872       [Social Welfare]         58     None   
                            hr4872       [Social Welfare]         57     None   
                            hr4872       [Social Welfare]         58     None   
                            hr4872       [Social Welfare]         56     None   
                            hr4872       [Social Welfare]         57     None   
                            hr4872       [Social Welfare]         58     None   
                            hr4872       [Social Welfare]         58  